In [219]:
### Cell 0 ###
### Imports ###

import numpy as np
import tensorflow as tf

In [220]:
### Cell 1 ###
### Data generation and normalization ###

train_size = 3000
valid_size = 1000
test_size = 1000

# y = k * x + b
k = 2
b = 5

# It is important to properly shuffle the training data -- failure to do so may result in SGD diverging.

# From 0 to 10, uniformly distributed values.
train_x = (10 * np.random.rand(train_size)).astype(np.float32).reshape((train_size, -1))
train_y = (k * train_x + b + 0.01 * np.random.randn()).reshape((train_size, -1))
valid_x = (10 * np.random.rand(valid_size)).astype(np.float32).reshape((valid_size, -1))
valid_y = (k * valid_x + b + 0.01 * np.random.randn()).reshape((valid_size, -1))
test_x = (10 * np.random.rand(test_size)).astype(np.float32).reshape((test_size, -1))
test_y = (k * test_x + b + 0.01 * np.random.randn()).reshape((test_size, -1))

# Rescale to [-1; 1]
def normalize(arr):
    return (arr - np.mean(arr)) / (np.max(arr) - np.min(arr))
train_x = normalize(train_x)
train_y = normalize(train_y)
valid_x = normalize(valid_x)
valid_y = normalize(valid_y)
test_x = normalize(test_x)
test_y = normalize(test_y)

In [221]:
### Cell 2 ###
### TensorFlow Graph definition ###

batch_size = 300
learning_rate = 0.00001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, train_x.shape[1]))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, train_y.shape[1]))
    tf_valid_dataset = tf.constant(valid_x)
    tf_test_dataset = tf.constant(test_x)
    
    weights = tf.Variable([[0.]])
    biases = tf.Variable([[0.]])
  
    # Training step computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.nn.l2_loss(logits - tf_train_labels) / batch_size
  
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    train_prediction = tf.matmul(tf_train_dataset, weights) + biases
    valid_prediction = tf.matmul(tf_valid_dataset, weights) + biases
    test_prediction = tf.matmul(tf_test_dataset, weights) + biases

In [223]:
### Cell 3 ###
### Run and evaluate ###

num_steps = 500

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        # Training data are randomized.
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        batch_data = train_x[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 20 == 0):
            print("Batch loss at step %d: %.8f" % (step, l))
            print("Validation loss at step %d: %.8f\n" % (step, ((sum((valid_prediction.eval() - valid_y) ** 2)) / valid_size / 2.)))
    print("Test loss: %.8f" % (sum((test_prediction.eval() - test_y) ** 2) / test_size / 2.))

Batch loss at step 0: 0.04324540
Validation loss at step 0: 0.04231153

Batch loss at step 20: 0.04041901
Validation loss at step 20: 0.04231017

Batch loss at step 40: 0.04135164
Validation loss at step 40: 0.04230875

Batch loss at step 60: 0.03924545
Validation loss at step 60: 0.04230731

Batch loss at step 80: 0.04350670
Validation loss at step 80: 0.04230596

Batch loss at step 100: 0.04647830
Validation loss at step 100: 0.04230455

Batch loss at step 120: 0.04147103
Validation loss at step 120: 0.04230314

Batch loss at step 140: 0.04134494
Validation loss at step 140: 0.04230171

Batch loss at step 160: 0.03719819
Validation loss at step 160: 0.04230027

Batch loss at step 180: 0.04323247
Validation loss at step 180: 0.04229889

Batch loss at step 200: 0.04040691
Validation loss at step 200: 0.04229748

Batch loss at step 220: 0.04133923
Validation loss at step 220: 0.04229609

Batch loss at step 240: 0.03923370
Validation loss at step 240: 0.04229467

Batch loss at step 260: 